## Molecular Desing


In [ ]:
#| code-line-numbers: '|6|9'
#| echo: true

import numpy as np
import matplotlib.pyplot as plt

r = np.arange(0, 2, 0.01)
theta = 2 * np.pi * r
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, r)
ax.set_rticks([0.5, 1, 1.5, 2])
ax.grid(True)
plt.show()

## The problem

## Representing molecules

## Predicting properties

## The problem

Find extreme of objective function 

\begin{equation*}
\arg\max_{x \in \mathcal{S}} f(x)
\end{equation*}

* No closed form for $f(x)$

* $f(x)$ is expensive to evaluate

* Possibly discrete structured domain $\mathcal{S}$ (e.g. $\mathcal{S} = [k]^p$)

* We can query at $x$  and obtain a (possibly noisy) evaluation of $f(x)$

## Goal

Get good estimates of global maximum, with **few objective function evaluations**

## Bayesian Optimization (in a nutshell)

Imagine we have access to: $\mathcal{D}_{1:T} = \lbrace x_t, y_t \rbrace_{t=1}^T$
where $y_t  \equiv y(x_t) \equiv f(x_t) + \epsilon$, $\epsilon \sim \mathcal{N}(0, \sigma^2)$
<br>
<br>
Q: Where to evaluate next? 

A: Location with max Expected Utility^[Following Bayesian Decision Theory principles]

## Bayesian Optimization (in a nutshell)

To do this:

1. Create **probabilistic model** of response given covariates

2. Given prior knowledge about $f(x)$ and evidence coming from $\mathcal{D}_{1:T}$, summarise our **beliefs about next sample result** through PPD

\begin{equation*}
p[y_{T+1} | x_{T+1}, \mathcal{D}_{1:T}] 
\end{equation*}

## Bayesian Optimization (in a nutshell)

3. Find

\begin{equation*}
x^*_{T+1} = \arg\max_{x_{T+1} \in \mathcal{S}}  \mathbb{E}_{y_{T+1} | x_{T+1}, \mathcal{D}_{1:T}} [u(y_{T+1}, x_{T+1} )]
\end{equation*}

## Exploration - Exploitation

Utility function

## Bayesian Optimization - Difficulties

$\mathcal{S}$ could be a combinatorial search space (or even more complex!). We need:

1. Suitable models of response given covariates

2. Solve a combinatorial optimization in Step 3.

## SBBO

We propose **Simulation Based Bayesian Optimization (SBBO)**: an approach to 2 that just requires **sampling from PPD** allowing us to use a wide variaty of models in 1


## SBBO 

The EU is:

\begin{eqnarray*}
\Psi(x) \equiv \int u(y, x ) \cdot \pi(y \vert x, \mathcal{D}_{1:T} ) d y
\end{eqnarray*}

Given **non-negative** and **bounded** utility, recast EU maximization as a simulation from 

\begin{eqnarray*}
g(x, y) \propto u(y, x ) \cdot \pi(y \vert x, \mathcal{D}_{1:T} ) 
\end{eqnarray*}

**NOTE:** mode of marginal in $x$ is $x^*_{T+1}$!

## SBBO 

* We could simulate from $g(y, x)$ and find the mode... just limited for low dimensional $x$

## SBBO 

* Alternative: **auxiliary distribution**

\begin{eqnarray*}
g_H(x, y_1, \dots, y_H) \propto \prod_{h=1}^H u(y^h, x ) \cdot \pi(y^h \vert x, \mathcal{D}_{1:T} ) 
\end{eqnarray*}

for positive integer $H$. Marginal in $x$ 

\begin{eqnarray*}
g_H(x) \propto \Psi(x)^H
\end{eqnarray*}

## SBBO 

Muller et. al. (2004): inhomogeneus MCMC simulation from $g_H(y, x)$ with increasing $H=H_n$ such that **stationary distribution** for fixed $H$ is $g_H$ converges to uniform over set of expected utility maxmizers

## SBBO - Gibbs

Let's define

\begin{eqnarray*}
g_H(x, y_1, \dots, y_H) \propto \exp \left \lbrace
\sum_{h=1}^H \log[ u(y^h, x ) ] + \log [\pi(y^h \vert x, \mathcal{D}_{1:T}) ] \right \rbrace 
\end{eqnarray*}

## SBBO - Gibbs

Recall $x \in [k]^p$ is a $p$-dimensional vector of $k$-levels categorical variables. Then:

\begin{eqnarray*}
g_H(x_q \vert \cdot) \propto \exp \left \lbrace
\sum_{h=1}^H \log[ u(y^h, x_q \cup x_{-q}  ) ] + \log [\pi(y^h \vert x_q \cup x_{-q}, \mathcal{D}_{1:T}) ] \right \rbrace 
\end{eqnarray*}

*Softmax* over  $\sum_{h=1}^H \log[ u(y^h, x_q \cup x_{-q}  ) ] + \log [\pi(y^h \vert x_q \cup x_{-q}, \mathcal{D}_{1:T}) ]$ 
for every level $x_q$!

## SBBO - Gibbs

Assume current state of chain is $x, y_1, \dots, y_H$. Iterate

1. For $q=1, \dots, p$, sample $x_q \sim g_H(x_q \vert \cdot)$ 
<br>
<br>
2. For $h=1, \dots, H$, sample $y_h \sim g_H(y_h \vert \cdot)$, using a Metropolis-Hastings step. 
<br>
<br>
3. Increase  $H$ according to chosen cooling schedule.

## SBBO - Gibbs

We need to evaluate $\sum_{h=1}^H \log[ u(y^h, x_q \cup x_{-q}  ) ] + \log [\pi(y^h \vert x_q \cup x_{-q}, \mathcal{D}_{1:T}) ]$ ... 
<br>
<br>
we need closed form **log posterior predictive distribution**

## SBBO - Metropolis

* Update $y^h$ and $x_q$ at the same time

* Use a symmetric proposal for $x_q$

* Use $\pi(y^h \vert x, \mathcal{D}_{1:T})$ as proposal for $y_h$

## SBBO - Metropolis

Assume current state of chain is $x, y_1, \dots, y_H$.

Define $v = \frac{1}{H} \sum_h \log u(x, y_h)$. Iterate:

1. For $q=1, \dots, p$:

    * Sample $\tilde{x}_q$ from symmetric proposal

    * Sample $\tilde{y}_h \sim \pi(y^h \vert \tilde{x}_q \cup x_{-q}, \mathcal{D}_{1:T} )$ for $h=1, \dots, H$

    * Evaluate $\tilde{v} = \frac{1}{H} \sum_h \log u(\tilde{x}_q \cup x_{-q}, \tilde{y}_h)$

    * With probability $\min \left[1, \exp (H \tilde{v} - H v)\right]$ set $v = \tilde{v}$ and $x = \tilde{x}_q \cup x_{-q}$

2. Increase $H$ according to chosen cooling schedule

## SBBO - Implementation details

* We run the previous algorithm increasing $H$ until certain value.

* Last generated $x$ is the new evaluation

* We propose several probabilistic models of response given covariates for which we have sampling access to their PPD

## SBBO - Tanimoto GP

* Uncertainty on $f(x)$ modelled through Gaussian Process

* $x \in \lbrace 0, 1 \rbrace^p$. Kernel function:

\begin{equation*}
k(x, x') = \frac{x \cdot x'}{\Vert x \Vert^2 + \Vert x' \Vert^2 - x \cdot x'}
\end{equation*}

* **PPD is Gaussian with certain mean and variance analytically available**

## SBBO - Sparse Bayesian linear regression

As in Baptista and Poloczek (2018):
\begin{eqnarray*}
&& y = \alpha_0 + \sum_j \alpha_j x_j + \sum_{i,j>i} \alpha_{ij} x_i x_j + \epsilon\\
%
&& \alpha_k \vert \beta_k, \tau, \sigma^2 \sim \mathcal{N}(0, \beta_k^2 \tau^2 \sigma^2)\\
%
&& \beta_k, \tau \sim \mathcal{C}^+(0, 1)\\
%
&& P(\sigma^2) \propto \sigma^{-2}
\end{eqnarray*}

* **PPD accesible through MCMC (Gibbs sampler)**


## SBBO - NGBoost

* Output given covariates modelled through
$y \vert x \sim P_\theta (x)$

* Where $\theta(x)$ are obtained through an additive combination of $M$ base learners and an initial $\theta^{(0)}$

$$
\theta = \theta^{(0)} - \eta \sum_{m=1}^M \rho^{(m)}\cdot f^{(m)} (x)
$$

* Learners are trained to minimize a **proper scoring rule** using a refinment of **gradient boosting**

## SBBO - NGBoost

* Any base learner can be used

* Base learners used: **shallow decision trees** and **lasso linear regressions** 

* **PPD directly accesible**

## Transition

General algorithmic things

* Initial sample size

## Binary Quadratic Problem

* We want to maximize

\begin{equation*}
x^T Q x - \lambda \Vert x \Vert_1
\end{equation*}

over $\lbrace 0,1 \rbrace^d$, with $d=10$.

* $Q$ is random matrix with indep. Gaussian entries, multiplied element-wise by matrix $K_{ij} = \exp[-(i-j)^2 / L_c^2]$

## Result

![](iiia23_files/BQP.png){ width="800" height="600" style="display: block; margin: 0 auto"}

::: footer
Binary Quadratic Problem 
:::


## Contamination Problem

* Food supply with $d$ stages that maybe contaminated

* $Z_i$ denotes fraction of food contaminated at $i$-th stage

* At stage $i$, prevention effort (with cost $c_i$) can be made $(x_i = 1)$ decreasing contamination a random rate $\Gamma_i$

* If no prevention is taken $(x_i = 0)$, contamination spreads with random rate $\Lambda_i$

\begin{equation*}
Z_i = \Lambda_i (1-x_i)(1 - Z_{i-1}) + (1 - \Gamma_i x_i) Z_{i-1}
\end{equation*}

![](iiia23_files/clem.jpeg){width="400" height="150" style="display: block; margin: 0 auto"}


## Contamination Problem - Goal

* Decide for each stage whether to interven or not to minimize cost

* Ensuring fraction of cont. food does not exceed $U_i$ with probability at least $1-\epsilon$

* Lagrangian relaxation

\begin{equation*}
\arg\min_x \sum_{i=1}^d \left[ c_i x_i + \frac{\rho}{T} \sum_{k=1}^T 1_{\lbrace Z_{ik} > U_i \rbrace} - (1- \epsilon)\right] + \lambda \Vert x \Vert_1
\end{equation*}


## Results

![](iiia23_files/CON.png){width="800" height="600" style="display: block; margin: 0 auto"}

::: footer
Contamination Problem 
:::

## Results - Why?

![](iiia23_files/CON_acc50.png){width="800" height="600" style="display: block; margin: 0 auto"}

::: footer
Contamination Problem 
:::

## Results - Why?

![](iiia23_files/CON_acc200.png){width="800" height="600" style="display: block; margin: 0 auto"}

::: footer
Contamination Problem 
:::

## Results - Why?

![](iiia23_files/CON_acc400.png){width="800" height="600" style="display: block; margin: 0 auto"}

::: footer
Contamination Problem 
:::




## Conclusions